In [58]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib

csv = pd.read_csv("../data/turnover.csv", encoding='utf-8')

csv["entree"] = pd.to_datetime(csv["entree"])
csv["sortie"] = pd.to_datetime(csv["sortie"])
csv["mandat_debut"] = pd.to_datetime(csv["mandat_debut"])

csv["contrat_fini"] = (csv["sortie"] == max(csv["sortie"]))
csv["duree_mandat"] = max(csv["sortie"]) - csv["mandat_debut"]
csv["duree_mandat"] = csv["duree_mandat"].apply(lambda a: int(a.total_seconds()/60/60/24))

In [59]:
csv

,parlementaire,nom_parlementaire,prénom_parlementaire,sexe_parlementaire,collaborateur,nom_collaborateur,prénom_collaborateur,sexe_collaborateur,url_nosdeputes_api,url_institution,...,url_an,id_an,slug,url_nosdeputes,nb_mandats,entree,sortie,nombredejours,contrat_fini,duree_mandat
0,Julien Dive,Dive,Julien,H,M. Nicolas Pelissier-Combescure,Nicolas,Pelissier-Combescure,H,https://www.nosdeputes.fr/julien-dive/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,712015,julien-dive,https://www.nosdeputes.fr/julien-dive,2,2017-07-22 13:01:40,2017-07-25 01:15:29,2,False,372
1,Éric Alauzet,Alauzet,Éric,H,M. Guillaume Travaglini,Guillaume,Travaglini,H,https://www.nosdeputes.fr/eric-alauzet/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,605963,eric-alauzet,https://www.nosdeputes.fr/eric-alauzet,1,2017-07-22 13:01:40,2018-06-28 22:00:00,341,True,372
2,Jean-Noël Barrot,Barrot,Jean-Noël,H,M. Edgar Tilly,Edgar,Tilly,H,https://www.nosdeputes.fr/jean-noel-barrot/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,721836,jean-noel-barrot,https://www.nosdeputes.fr/jean-noel-barrot,1,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372
3,Jean-Noël Barrot,Barrot,Jean-Noël,H,M. Edgar Tilly,Tilly,Edgar,H,https://www.nosdeputes.fr/jean-noel-barrot/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,721836,jean-noel-barrot,https://www.nosdeputes.fr/jean-noel-barrot,1,2017-10-19 01:15:23,2018-01-26 02:15:27,99,False,372
4,Marie-Noëlle Battistel,Battistel,Marie-Noëlle,F,Mme Dorothée Collet,Dorothée,Collet,F,https://www.nosdeputes.fr/marie-noelle-battist...,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,342415,marie-noelle-battistel,https://www.nosdeputes.fr/marie-noelle-battistel,1,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372
5,Philippe Berta,Berta,Philippe,H,M. REGIS VEZON,Regis,Vezon,H,https://www.nosdeputes.fr/philippe-berta/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,719488,philippe-berta,https://www.nosdeputes.fr/philippe-berta,1,2017-07-22 13:01:40,2018-03-31 01:15:48,251,False,372
6,Alexis Corbière,Corbière,Alexis,H,M. Erol Yolal,Erol,Yolal,H,https://www.nosdeputes.fr/alexis-corbiere/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,721210,alexis-corbiere,https://www.nosdeputes.fr/alexis-corbiere,1,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372
7,Laurence Dumont,Dumont,Laurence,F,Mme Bérénice Victoire-Féron,Bérénice,Victoire-Féron,F,https://www.nosdeputes.fr/laurence-dumont/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,1198,laurence-dumont,https://www.nosdeputes.fr/laurence-dumont,1,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372
8,Marietta Karamanli,Karamanli,Marietta,F,Mme Delphine Froger,Delphine,Froger,F,https://www.nosdeputes.fr/marietta-karamanli/xml,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,335054,marietta-karamanli,https://www.nosdeputes.fr/marietta-karamanli,2,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372
9,Fabrice Le Vigoureux,Le Vigoureux,Fabrice,H,M. Augustin Le Coutour,Augustin,Le Coutour,H,https://www.nosdeputes.fr/fabrice-le-vigoureux...,http://www2.assemblee-nationale.fr/deputes/fic...,...,http://www2.assemblee-nationale.fr/deputes/fic...,719006,fabrice-le-vigoureux,https://www.nosdeputes.fr/fabrice-le-vigoureux,1,2017-07-22 13:01:40,2017-07-26 01:15:28,3,False,372


In [60]:
nb_collaborateurs = csv.groupby(["parlementaire"])[['collaborateur']].count().rename(columns={"collaborateur": "nb_collaborateurs"})

In [73]:
duree_moy_collaborateurs = csv.groupby(["parlementaire"])[['nombredejours']].mean().rename(columns={"nombredejours": "duree_moyenne"})
duree_moy_collaborateurs = duree_moy_collaborateurs.merge(nb_collaborateurs, on="parlementaire").merge(csv.groupby(["parlementaire"])[['duree_mandat']].first(), on="parlementaire").merge(csv.groupby(["parlementaire"])[['nombredejours']].max(), on="parlementaire").sort_values("duree_moyenne")
duree_moy_collaborateurs["ratio_contrat_mandat"] = duree_moy_collaborateurs["duree_moyenne"] / duree_moy_collaborateurs["duree_mandat"]
duree_moy_collaborateurs = duree_moy_collaborateurs.rename(columns={"nombredejours": "contrat le plus long"})

In [83]:
duree_moy_collaborateurs[duree_moy_collaborateurs["ratio_contrat_mandat"] < 0.7].sort_values("contrat le plus long")

,duree_moyenne,nb_collaborateurs,duree_mandat,contrat le plus long,ratio_contrat_mandat
parlementaire,,,,,
José Evrard,101.428571,7,372,154,0.272657
Patrick Vignal,75.384615,13,372,170,0.202647
Anissa Khedher,78.875000,8,372,170,0.212030
Claire Guion-Firmin,170.000000,2,372,187,0.456989
Loïc Dombreval,161.750000,4,372,189,0.434812
Marie Lebec,185.750000,4,372,197,0.499328
Valérie Petit,129.714286,7,372,204,0.348694
Sonia Krimi,80.666667,12,372,209,0.216846
Sandra Marsaud,97.571429,7,372,211,0.262289


In [84]:
duree_moy_sexe = csv.groupby(["parlementaire", "sexe_parlementaire", "sexe_collaborateur"])[['nombredejours', 'sexe_collaborateur']].mean()

In [85]:
nb_collab_sexe = csv.groupby(["parlementaire", "sexe_collaborateur"])[['nombredejours', 'sexe_collaborateur']].size().to_frame("nombredecollab")

In [86]:
duree_moy_sexe["nombredecollab"] = nb_collab_sexe["nombredecollab"]

In [87]:
duree_moy_sexe

nombredejours  \
parlementaire              sexe_parlementaire sexe_collaborateur                  
Adrien Morenas             H                  F                      327.000000   
                                              H                      329.000000   
Adrien Quatennens          H                  H                      318.333333   
Adrien Taquet              H                  F                      247.000000   
                                              H                      156.666667   
Agnès Firmin Le Bodo       F                  F                      329.250000   
Agnès Thill                F                  F                      245.000000   
                                              H                      233.000000   
Aina Kuric                 F                  F                      318.500000   
                                              H                      197.333333   
Alain Bruneel              H                  F                      281.333333   
                                              H                      212.000000   
Alain David                H                  F                      309.000000   
                                              H                      135.333333   
Alain Perea                H                  F                      154.500000   
                                              H                      187.250000   
Alain Ramadier             H                  F                      292.000000   
                                              H                      316.500000   
Alain Tourret              H                  F                      341.000000   
                                              H                      247.333333   
Albane Gaillot             F                  F                      180.000000   
                                              H                      165.000000   
Alexandra Louis            F                  F                       75.500000   
                                              H                      265.000000   
Alexandra Valetta Ardisson F                  F                      139.833333   
                                              H                       77.000000   
Alexandre Freschi          H                  F                      111.333333   
                                              H                       43.500000   
Alexandre Holroyd          H                  F                      260.000000   
                                              H                      260.000000   
...                                                                         ...   
Élodie Jacquier-Laforge    F                  F                      312.500000   
                                              H                      182.000000   
Émilie Bonnivard           F                  F                      131.250000   
                                              H                      208.000000   
Émilie Cariou              F                  F                      337.000000   
                                              H                      283.250000   
Émilie Chalas              F                  F                      195.000000   
                                              H                      294.000000   
Émilie Guerel              F                  F                      266.333333   
                                              H                       56.333333   
Éric Alauzet               H                  F                      139.000000   
                                              H                      192.400000   
Éric Bothorel              H                  F                      145.000000   
                                              H                      341.000000   
Éric Ciotti                H                  F                      325.500000   
                                              H                       60.500000   